## 서울특별시 다산콜센터(☎120)의 주요 민원
* 서울특별시 다산콜센터(☎120)의 주요 민원(자주 묻는 질문)에 대한 답변정보
* https://opengov.seoul.go.kr/civilappeal/list

In [1]:
# 필요한 라이브러리들을 불러옵니다.
# pandas : 파이썬에서 사용할 수 있는 엑셀과 유사한 데이터분석 도구입니다.
# requests : 매우 작은 브라우저로 웹사이트의 내용과 정보를 불러옵니다.
# BeautifulSoup : request로 가져온 웹사이트의 html 태그를 찾기 위해 사용합니다.
# time : 한 번에 많은 양의 데이터를 가져오게 되면 서버에서 부담을주기 때문에 시간 간격을 두고 가져오기 위해 사용합니다.

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import time

In [2]:
# 수집해온 파일("seoul-120-list.csv")을 읽어옵니다.
# df
df = pd.read_csv('seoul-120-list.csv')
print(df.shape)

(2450, 5)


## 일부 데이터를 우선 수집
* 현업에서는 모든 데이터를 수집해야 하지만 모든 데이터 수집시 서버에 부담이 될 수 있으며 오래 걸립니다.
* 또 동작하지 않거나 잘못 수집을 했다면 처음부터 다시 수집해야 하기 때문에 일부 데이터만을 통해 먼저 수집해 보는 것을 추천합니다.
* 일부 데이터를 먼저 수집해 보고 데이터가 필요할 때 전체를 수집해 봅니다.

In [3]:
# head 로 일부 데이터만 가져와서 먼저 수집해 봅니다.
df = df.head(10)
df

,번호,제목,생산일,조회수,내용번호
0,2470,다자녀가정 실내 바닥매트 지원,2021-08-17,156,25670204
1,2469,[서울산업진흥원] 서울메이드란?,2021-06-29,675,23194045
2,2468,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,712,22904492
3,2467,마포 뇌병변장애인 비전센터,2021-03-12,840,22477798
4,2466,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,1177,21212235
5,2465,"[성동구] 반려동물 ""무료상담실"" 운영 안내",2020-08-31,905,21095911
6,2464,서울시 '우리동네꼼꼼육아정보' 책자,2019-10-23,1569,18955676
7,2463,서울시 사회서비스원은 무엇을 하는 곳인가요?,2019-07-22,1647,18300944
8,2462,"[종료사업] S-Taxi 앱 (에스택시, 서울택시승차앱) 시범운영",2019-06-03,1631,17949033
9,2461,장기요양 이동지원서비스(모두타 돌봄택시),2019-05-27,2142,17896364


## 특정 내용 읽어오기
* 내용의 URL을 확인합니다.
* requests를 통해 내용에 접근합니다.
* BeautifulSoup의 select를 사용해 내용이 있는 태그를 찾습니다.


In [4]:
# 내용 페이지의 주소를 url 변수에 담아줍니다.
# 웹페이지의 결과를 받아옵니다.
# BeautifulSoup을 통해 lxml로 파싱해 올 수 있도록 합니다.
# html 태그에서 "div.line-all"의 0번째 값을 선택(select)하여 text를 확인합니다. 

url = "https://opengov.seoul.go.kr/civilappeal/view/?nid=23194045"
print(url)

# response
header = {'user_agent':'Mozilla/5.0'}
r = requests.get(url,headers=header)
print(r.status_code)

# html
html = bs(r.text,'lxml')

# content
content = html.select('div.line-all')[0].text
content

https://opengov.seoul.go.kr/civilappeal/view/?nid=23194045
200


'\n\xa0서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해 만들어진 라이프스타일 플랫폼 브랜드로 서울이 만들고 세계가 함께 쓰는 브랜드 입니다.\xa0 \xa0 - 서울의 감성을 담은 기업과 협업하여 브랜드 상품을 발굴. \xa0 -\xa0사용을 원하는 중소기업에게 서울메이드 브랜드 제공 등 \xa0 -\xa0 관련홈페이지 :\xa0서울메이드 (https://seoulmade.com/) \xa0※ 상세문의사항 :\xa0서울산업진흥원\xa0브랜드전략팀 02-2222-3764 '

In [19]:
table = pd.read_html(r.text)[-1]
table

,0,1,2,3
0,원본시스템,다산콜센터,제공부서,서울산업진흥원
1,작성자(책임자),120다산콜재단,생산일,2021-06-29
2,관리번호,D0000042894548,분류,경제


In [13]:
table.iloc[:,:2]

,0,1
0,원본시스템,다산콜센터
1,작성자(책임자),120다산콜재단
2,관리번호,D0000042894548


In [6]:
table[[2,3]].set_index(2).T

2,제공부서,생산일,분류
3,서울산업진흥원,2021-06-29,경제


In [7]:
# 해당 문의가 어떤 분류에 해당되는지 알기 위해 분류를 수집합니다.
def get_desc(response):
    """ 분류 수집하기 """
    table = pd.read_html(response.text)[1]
    t01 = table[[0,1]].set_index(0).T
    t02 = table[[2,3]].set_index(2).T
    t02.index = t01.index
    df = pd.concat((t01,t02),axis=1)
    return df

get_desc(r)

,원본시스템,작성자(책임자),관리번호,제공부서,생산일,분류
1,다산콜센터,120다산콜재단,D0000042894548,서울산업진흥원,2021-06-29,경제


In [88]:
# 해당 문의가 어떤 분류에 해당되는지 알기 위해 분류를 수집합니다.
def get_desc(response):
    """ 분류 수집하기 """
    html = bs(response.text)
    tables = html.select('table')
    table = pd.read_html(str(tables[-1]))[0]
    t01 = table[[0,1]].set_index(0).T
    t02 = table[[2,3]].set_index(2).T
    t02.index = t01.index
    df = pd.concat((t01,t02),axis=1)
    return df

get_desc(r)

,원본시스템,작성자(책임자),관리번호,제공부서,생산일,분류
1,다산콜센터,120다산콜재단,D0000042894548,서울산업진흥원,2021-06-29,경제


In [23]:
html.select('div.line-all')[0]

<div class="line-all" style="overflow: auto">
<p style="line-height: 150%;"><span style="color: rgb(0, 0, 0);"><span id="aGi_StartCaret"></span> 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해 만들어진 라이프스타일 플랫폼 브랜드로 서울이 만들고 세계가 함께 쓰는 브랜드 입니다. </span><br style="margin: 0px; padding: 0px;"/> <span style="color: rgb(0, 0, 0);">  - 서울의 감성을 담은 기업과 협업하여 브랜드 상품을 발굴.</span><br style="margin: 0px; padding: 0px;"/> <span style="color: rgb(0, 0, 0);">  - 사용을 원하는 중소기업에게 서울메이드 브랜드 제공 등</span><br style="margin: 0px; padding: 0px;"/> <span style="color: rgb(0, 0, 0);">  -  관련홈페이지 : 서울메이드 (<a href="https://seoulmade.com/" target="_blank" title="">https://seoulmade.com/)</a></span><br style="margin: 0px; padding: 0px;"/> <span style="color: rgb(0, 0, 0);"> ※<strong> 상세문의사항 : 서울산업진흥원 브랜드전략팀 02-2222-3764</strong></span><span style="color: rgb(0, 0, 255);"></span></p> </div>

## 내용 수집 함수 만들기

In [43]:
# 위의 전체 과정을 함수로 만들어 봅니다.

def get_view_page(view_no):
    """ 
    내용과 분류를 수집하는 함수 만들기
    """
    
    # URL 및 데이터 세팅
    url = f'https://opengov.seoul.go.kr/civilappeal/view/?nid={view_no}'
    header = {'user_agent':'Mozilla/5.0'}
    r = requests.get(url, headers=header)
    html = bs(r.text)
    
    # 문서 본문 가져오기
    content = html.select('#content > div > div.view-content.view-content-article > div:nth-child(2) > div')[0].get_text()
    
    # 문서 정보 가져오기
    df_desc = get_desc(r)
    df_desc['내용'] = content
    time.sleep(0.01)
    
    return df_desc

get_view_page(23194045)

,원본시스템,작성자(책임자),관리번호,제공부서,생산일,분류,내용
1,다산콜센터,120다산콜재단,D0000042894548,서울산업진흥원,2021-06-29,경제,\n 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해 만들어진 라이프스타일...


In [ ]:
# 없는 데이터를 불러왔을 때 오류가 나는지 확인
# view_no가 22904492 인 것을 get_view_page()로 확인해 봅니다.
get_view_page(view_no=22904492)

## 전체 내용 가져오기
* <font color="red">실습을 위해 위에서 일부 데이터만 가져왔습니다.</font>
* 전체 데이터 수집시에서 전체 수집한 데이터로 수집을 진행하면 됩니다.
* 전체 데이터 수집시에는 서버에 부담이 가지 않도록 time.sleep()을 통해 간격을 두고 수집해 주세요.

In [45]:
# tqdm.notebook 의 tqdm 을 통해 수집 진행상태를 확인합니다.
# progress_apply 를 사용하면 진행상태를 확인하며 데이터를 가져올 수 있습니다.
from tqdm.notebook import tqdm
tqdm.pandas()
view = df['내용번호'].progress_map(get_view_page)

  0%|          | 0/10 [00:00<?, ?it/s]

In [50]:
df.head()

,번호,제목,생산일,조회수,내용번호
0,2470,다자녀가정 실내 바닥매트 지원,2021-08-17,156,25670204
1,2469,[서울산업진흥원] 서울메이드란?,2021-06-29,675,23194045
2,2468,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,712,22904492
3,2467,마포 뇌병변장애인 비전센터,2021-03-12,840,22477798
4,2466,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,1177,21212235


## 수집한 내용 확인하기

In [53]:
# 일부 데이터만 슬라이싱하여 살펴봅니다.
view[:4]

0       원본시스템  작성자(책임자)            관리번호            ...
1       원본시스템  작성자(책임자)            관리번호     제공부서   ...
2       원본시스템  작성자(책임자)            관리번호            ...
3       원본시스템  작성자(책임자)            관리번호            ...
Name: 내용번호, dtype: object

In [52]:
# 하나만 가져오면 데이터프레임 형태로 되어 있음을 확인합니다.
view[0]

,원본시스템,작성자(책임자),관리번호,제공부서,생산일,분류,내용
1,다산콜센터,120다산콜재단,D0000043274644,서울특별시 서초구 주민생활국 여성보육과,2021-08-17,복지,\n 업무개요 다자녀가정 아이들의 안전한 실내 활동과 층간소음 예방을 ...


## 하나로 병합하기

In [63]:
# 수집한 내용을 tolist() 를 통해 리스트로 변환 후 concat 으로 병합합니다.
# df_view
df_view = pd.concat(view.to_list())
df_view.reset_index(inplace=True, drop=True)
print(df_view.shape)
df_view.head()

(10, 7)


,원본시스템,작성자(책임자),관리번호,제공부서,생산일,분류,내용
0,다산콜센터,120다산콜재단,D0000043274644,서울특별시 서초구 주민생활국 여성보육과,2021-08-17,복지,\n 업무개요 다자녀가정 아이들의 안전한 실내 활동과 층간소음 예방을 ...
1,다산콜센터,120다산콜재단,D0000042894548,서울산업진흥원,2021-06-29,경제,\n 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해 만들어진 라이프스타일...
2,다산콜센터,120다산콜재단,D0000042557522,서울특별시 광진구 복지국 가정복지과,2021-05-13,복지,\n 업무개요 교통약자인 임산부와 영아가정 주민이 병원 진료를 받으러 갈 떄...
3,다산콜센터,120다산콜재단,D0000042114742,서울특별시 복지정책실 복지기획관 장애인복지정책과,2021-03-12,복지,\n ▣ 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학령기 이...
4,다산콜센터,120다산콜재단,D0000040829456,서울특별시 여성가족정책실 여성권익담당관,2020-09-16,복지,\n 업무개요 직장 내 성희롱 지원체계 사각지대에 있는 서울 시민이 안전하...


In [69]:
# 기존 데이터와 병합하여 내용이 함께 수집된 것을 확인합니다.
# df_detail
df_detail = pd.concat([df_view, df],axis=1)
print(df_detail.shape)
df_detail.head()

(10, 12)


,원본시스템,작성자(책임자),관리번호,제공부서,생산일,분류,내용,번호,제목,생산일,조회수,내용번호
0,다산콜센터,120다산콜재단,D0000043274644,서울특별시 서초구 주민생활국 여성보육과,2021-08-17,복지,\n 업무개요 다자녀가정 아이들의 안전한 실내 활동과 층간소음 예방을 ...,2470,다자녀가정 실내 바닥매트 지원,2021-08-17,156,25670204
1,다산콜센터,120다산콜재단,D0000042894548,서울산업진흥원,2021-06-29,경제,\n 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해 만들어진 라이프스타일...,2469,[서울산업진흥원] 서울메이드란?,2021-06-29,675,23194045
2,다산콜센터,120다산콜재단,D0000042557522,서울특별시 광진구 복지국 가정복지과,2021-05-13,복지,\n 업무개요 교통약자인 임산부와 영아가정 주민이 병원 진료를 받으러 갈 떄...,2468,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,712,22904492
3,다산콜센터,120다산콜재단,D0000042114742,서울특별시 복지정책실 복지기획관 장애인복지정책과,2021-03-12,복지,\n ▣ 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학령기 이...,2467,마포 뇌병변장애인 비전센터,2021-03-12,840,22477798
4,다산콜센터,120다산콜재단,D0000040829456,서울특별시 여성가족정책실 여성권익담당관,2020-09-16,복지,\n 업무개요 직장 내 성희롱 지원체계 사각지대에 있는 서울 시민이 안전하...,2466,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,1177,21212235


## 사용할 컬럼만 남기기

In [75]:
# 제목 뒤에 문서가 보이도록 컬럼의 순서를 조정합니다.
# 사용할 컬럼만 인덱싱 합니다.
# 다음의 순서가 되게 합니다. '번호', '분류', '제목', '내용', '내용번호'
result = df_detail[['번호', '분류', '제목', '내용', '내용번호']]
result.head()

,번호,분류,제목,내용,내용번호
0,2470,복지,다자녀가정 실내 바닥매트 지원,\n 업무개요 다자녀가정 아이들의 안전한 실내 활동과 층간소음 예방을 ...,25670204
1,2469,경제,[서울산업진흥원] 서울메이드란?,\n 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해 만들어진 라이프스타일...,23194045
2,2468,복지,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",\n 업무개요 교통약자인 임산부와 영아가정 주민이 병원 진료를 받으러 갈 떄...,22904492
3,2467,복지,마포 뇌병변장애인 비전센터,\n ▣ 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학령기 이...,22477798
4,2466,복지,위드유 서울 직장 성희롱.성폭력 예방센터,\n 업무개요 직장 내 성희롱 지원체계 사각지대에 있는 서울 시민이 안전하...,21212235


## 파일로 저장하고 확인하기

<img src="https://pandas.pydata.org/docs/_images/02_io_readwrite.svg">

In [76]:
# 저장할 파일명을 "seoul-120-sample.csv" 로 설정해 줍니다.
file_name = "seoul-120-sample.csv"

In [80]:
# csv 파일로 저장합니다.
result.to_csv(file_name,index=False)

In [82]:
# 저장이 잘 되었는지 csv 파일로 읽어옵니다.
pd.read_csv(file_name).head()

,번호,분류,제목,내용,내용번호
0,2470,복지,다자녀가정 실내 바닥매트 지원,\n 업무개요 다자녀가정 아이들의 안전한 실내 활동과 층간소음 예방을 ...,25670204
1,2469,경제,[서울산업진흥원] 서울메이드란?,\n 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해 만들어진 라이프스타일...,23194045
2,2468,복지,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",\n 업무개요 교통약자인 임산부와 영아가정 주민이 병원 진료를 받으러 갈 떄...,22904492
3,2467,복지,마포 뇌병변장애인 비전센터,\n ▣ 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학령기 이...,22477798
4,2466,복지,위드유 서울 직장 성희롱.성폭력 예방센터,\n 업무개요 직장 내 성희롱 지원체계 사각지대에 있는 서울 시민이 안전하...,21212235
